In [1]:
from matplotlib import pyplot as plt
import numpy as np

plt.style.use('ggplot')

In [2]:
class fileReader:
    def __init__(self, fileName, columnLabelsLine=5, numHeaderLines=8, delimiter=","):
        self.Data = np.genfromtxt(fileName, delimiter=delimiter, skip_header=numHeaderLines)
        with open(fileName) as infile:
            content = infile.readlines()
            row = content[columnLabelsLine]
        self.dataIdx = {label:i for i,label in enumerate(row.split(delimiter)) if label != ""}
    
    
    def getDataForPoints(self, nLine, markers):           
        result = np.zeros((3, len(markers)))
        for col, marker in enumerate(markers):
            if not marker in markers:
                print("Marker not found")
                return -1
            result[:, col] = self.Data[nLine, self.dataIdx[marker]: self.dataIdx[marker]+3]
        return result

In [3]:
def TransformationMatrix(points):
    P2P1 = points[:,1]-points[:,0]
    P3P1 = points[:,2]-points[:,0]
    
    xAxis = P2P1/np.linalg.norm(P2P1)
    yAxis_unNormalized = np.cross(P2P1, P3P1)
    yAxis = yAxis_unNormalized/np.linalg.norm(yAxis_unNormalized)
    zAxis = np.cross(xAxis, yAxis)
    origin = np.mean(points, axis=1)
    
    transformMat = np.column_stack((xAxis, yAxis, zAxis, origin))
    return np.row_stack((transformMat, np.array([0, 0, 0, 1])))

In [4]:
def computeFJC(parentMarker, childMarkers, fileName):
    file = fileReader(fileName)
    nRows = file.Data.shape[0]
    A = np.zeros((3*nRows, 6))
    b = np.zeros((3*nRows, 1))

    for i in range(nRows):
        parentGlobalVecs = file.getDataForPoints(i, parentMarker)
        parent_transformMat = TransformationMatrix(parentGlobalVecs)
    
        childGlobalVecs = file.getDataForPoints(i, childMarkers)
        child_transformMat = TransformationMatrix(childGlobalVecs)

        R1 = parent_transformMat[0:3, 0:3]
        R2 = child_transformMat[0:3, 0:3]
        A[3*i:3*(i+1), :] = np.concatenate((np.identity(3), -np.matmul(R1.T, R2)), axis=1);
    
        child_parent_pos_global = child_transformMat[0:3,3] - parent_transformMat[0:3,3]
        child_parent_pos = np.matmul(R1.T,child_parent_pos_global)
        b[3*i:3*(i+1), 0] = child_parent_pos

    sHJC_lHJC = np.linalg.lstsq(A,b, rcond=-1)[0]

    # Joint Center in Parent fixed frame
    sHJC = np.concatenate((sHJC_lHJC[0:3,0], np.array([1])))

    # Joint Center in Child fixed frame
    lHJC = np.concatenate((sHJC_lHJC[3:,0], np.array([1])))
    
    return sHJC, lHJC

In [5]:
#name = "Temuge"
#rHjcFileName = name+"_RightHJC.csv"
#rKjcfileName = name+"_RightKJC.csv"
#rAjfileName = name+"_RightAJC.csv"

#lHjcFileName = name+"_LeftHJC.csv"
#lKjcfileName = name+"_LeftKJC.csv"
#lAjfileName = name+"_LeftAJC.csv"

floorHeight = (56.0139+56.8536+55.5098+54.0312+55.4664+55.4844)/6 - 6.5

rHjcFileName = "RightCross1.csv"
#rAjfileName = "RAJC.csv"
lHjcFileName = "LeftCross1.csv"
#lAjfileName = "LAJC.csv"

pelvisMarkersR = ["RAsis", "LAsis", "RPsi"]
pelvisMarkersL = ["RAsis", "LAsis", "LPsi"]

femurRMarkers = ["RThigh3", "RThigh2", "RThigh1"]
#tibiaRMarkers = ["RAnkleOut", "RAnkleIn", "RTibia1"]
#feetRMarkers = ["R1Meta", "R5Meta", "RHeel"]

femurLMarkers = ["LThigh3", "LThigh1", "LThigh2"]
#tibiaLMarkers = ["LAnkleOut", "LAnkleIn", "LTibia1"]
#feetLMarkers = ["L1Meta", "L5Meta", "LHeel"]

sRHJC = computeFJC(pelvisMarkersR, femurRMarkers, rHjcFileName)[0]
#sRKJC = computeFJC(femurRMarkers, tibiaRMarkers, rKjcfileName)[0]
#sRAJC = computeFJC(tibiaRMarkers, feetRMarkers, rAjfileName)[0]

sLHJC = computeFJC(pelvisMarkersL, femurLMarkers, lHjcFileName)[0]
#sLKJC = computeFJC(femurLMarkers, tibiaLMarkers, lKjcfileName)[0]
#sLAJC = computeFJC(tibiaLMarkers, feetLMarkers, lAjfileName)[0]

In [6]:
file = fileReader("TPosePelvis2.csv")
nRows = file.Data.shape[0]
#nRows = 1
#out = np.zeros((nRows, 3*20))
out = np.zeros((nRows, 3*8))

for i in range(nRows):

    ## Right leg
    pelvisGlobalVecs_t = file.getDataForPoints(i, pelvisMarkersR)
    pelvis_transformMat_t = TransformationMatrix(pelvisGlobalVecs_t)
    rHJC_global = np.matmul(pelvis_transformMat_t, sRHJC)
    
    #femurRGlobalVecs_t = file.getDataForPoints(i, femurRMarkers)
    #femurR_transformMat_t = TransformationMatrix(femurRGlobalVecs_t)
    #rKJC_global = np.matmul(femurR_transformMat_t, sRKJC)
    rKJC_global = np.mean(file.getDataForPoints(i, ["RKneeOut", "RKneeIn"]), axis=1)
    
    #tibiaRGlobalVecs_t = file.getDataForPoints(i, tibiaRMarkers)
    #tibiaR_transformMat_t = TransformationMatrix(tibiaRGlobalVecs_t)
    #rAJC_global = np.matmul(tibiaR_transformMat_t, sRAJC)
    
    ## Left leg
    pelvisGlobalVecs_t = file.getDataForPoints(i, pelvisMarkersL)
    pelvis_transformMat_t = TransformationMatrix(pelvisGlobalVecs_t)
    lHJC_global = np.matmul(pelvis_transformMat_t, sLHJC)
    
    #femurLGlobalVecs_t = file.getDataForPoints(i, femurLMarkers)
    #femurL_transformMat_t = TransformationMatrix(femurLGlobalVecs_t)
    #lKJC_global = np.matmul(femurL_transformMat_t, sLKJC)
    lKJC_global = np.mean(file.getDataForPoints(i, ["LKneeOut", "LKneeIn"]), axis=1)
    
    #tibiaLGlobalVecs_t = file.getDataForPoints(i, tibiaLMarkers)
    #tibiaL_transformMat_t = TransformationMatrix(tibiaLGlobalVecs_t)
    #lAJC_global = np.matmul(tibiaL_transformMat_t, sLAJC)
    
    # Derived markers
    midHJC_global = (rHJC_global+lHJC_global)/2
    midAsis = np.mean(file.getDataForPoints(i, ["RAsis", "LAsis"]), axis=1)
    midPsis = np.mean(file.getDataForPoints(i, ["RPsi", "LPsi"]), axis=1)
    midPelvis = (midAsis+midPsis)/2
    
    vin = lHJC_global-rHJC_global
    print(vin[2])
    #rAJC_projected = np.array(rAJC_global)
    #rAJC_projected[1] = floorHeight
    
    #lAJC_projected = np.array(lAJC_global)
    #lAJC_projected[1] = floorHeight
    
    #R1Meta_projected = file.getDataForPoints(i, ["R1Meta"]).reshape((3,))
    #R1Meta_projected[1] = floorHeight
    
    #R5Meta_projected = file.getDataForPoints(i, ["R5Meta"]).reshape((3,))
    #R5Meta_projected[1] = floorHeight
    
    #RHeel_projected = file.getDataForPoints(i, ["RHeel"]).reshape((3,))
    #RHeel_projected[1] = floorHeight
    
    #L1Meta_projected = file.getDataForPoints(i, ["L1Meta"]).reshape((3,))
    #L1Meta_projected[1] = floorHeight
    
    #L5Meta_projected = file.getDataForPoints(i, ["L5Meta"]).reshape((3,))
    #L5Meta_projected[1] = floorHeight
    
    #LHeel_projected = file.getDataForPoints(i, ["LHeel"]).reshape((3,))
    #LHeel_projected[1] = floorHeight
    
    #RMidMeta_projected = np.mean((R1Meta_projected, R5Meta_projected),axis = 0)
    #LMidMeta_projected = np.mean((L1Meta_projected, L5Meta_projected),axis = 0)
    
    #print("rHJC_global", rHJC_global)
    #print("lHJC_global", lHJC_global)
    #print("sRKJC_global", rKJC_global)
    #print("sLKJC_global", lKJC_global)
    #print("sRAJC_global", rAJC_global)
    #print("sLAJC_global", lAJC_global)
    #print(midHJC_global)
    #print(midAsis)
    #print(midPsis)
    #print(midPelvis)
    #print(sRAJC_projected)
    #print(sLAJC_projected)
    #print(R1Meta_projected)
    #print(R5Meta_projected)
    #print(RHeel_projected)
    
    #out[i,:] = np.concatenate((midAsis, midPsis, midPelvis,
    #                           rHJC_global[0:3], lHJC_global[0:3], midHJC_global[0:3],
    #                           rKJC_global[0:3], lKJC_global[0:3],  
    #                           rAJC_global[0:3], lAJC_global[0:3],
    #                           rAJC_projected[0:3], lAJC_projected[0:3],
    #                           R1Meta_projected, R5Meta_projected, RHeel_projected,
    #                           L1Meta_projected, L5Meta_projected, LHeel_projected, 
    #                           RMidMeta_projected, LMidMeta_projected), axis=0)
    out[i,:] = np.concatenate((midAsis, midPsis, midPelvis,
                               rHJC_global[0:3], lHJC_global[0:3], midHJC_global[0:3],
                               rKJC_global[0:3], lKJC_global[0:3]), axis=0)

np.savetxt("vin.csv", out, delimiter=",")

-188.12167688246
-188.1225675820682
-188.12359536929915
-188.12439219962084
-188.12528778834783
-188.12593591282786
-188.1265759880983
-188.12704406486486
-188.12755929725432
-188.1279514294871
-188.12831269207504
-188.12869851023626
-188.12901055090032
-188.1292644076742
-188.12915783300957
-188.12932886786137
-188.12904459120915
-188.12909905901452
-188.12894979530415
-188.12863756712093
-188.1282109220305
-188.127885153517
-188.12739044544384
-188.12667130978815
-188.12577645999556
-188.12490842625846
-188.12383611351555
-188.12245955426948
-188.12123947664622
-188.11978857349914
-188.11836994727707
-188.1171615694572
-188.11604490195805
-188.1151251656931
-188.1142079440158
-188.1135572160946
-188.11281883602814
-188.11253669384303
-188.1121894680534
-188.11188557805514
-188.11156057440826
-188.111415310202
-188.11127239957284
-188.1113900384908
-188.11129128060162
-188.1111658340855
-188.11093018489066
-188.11099338465758
-188.11092911429319
-188.1106417973598
-188.11040770566035


-188.1192936142278
-188.1190024514541
-188.11858998470035
-188.11817516877474
-188.1180178807856
-188.1175651842081
-188.11691003648752
-188.11636057650577
-188.11567770516194
-188.11490974044898
-188.1142609814407
-188.11357496772268
-188.1126899648553
-188.11174922179072
-188.1111203665534
-188.1107070092662
-188.11023773866134
-188.10963152269298
-188.10893011961744
-188.10875238271916
-188.10808164472493
-188.10760812912616
-188.10693896482198
-188.1064164772805
-188.10579578697389
-188.10489956461709
-188.10399787581696
-188.10337357810846
-188.10245836842358
-188.10126720134525
-188.1002564774646
-188.0991635931308
-188.09802055800168
-188.0967519500965
-188.09538195948676
-188.09426625884535
-188.09291419031865
-188.09167983207905
-188.090594377871
-188.08950395346017
-188.0884752455852
-188.0873758714052
-188.0868098903714
-188.08627172924827
-188.08597908968585
-188.0858242602518
-188.08589171319204
-188.0860152160576
-188.08616358838313
-188.08632139679395
-188.0862000788672


In [7]:
midAsis = np.array([0.00696365,0.0103228,0.0])
misPsis = np.array([-0.153155,0.0426324,0.0])

midPelvis = np.mean((midAsis, misPsis), axis=0)

#R1Meta = np.array([0.174285,-0.0147377,-0.0298436])
#R5Meta = np.array([0.137032,-0.0147377,0.0412118])
#RmidMeta = np.mean((R1Meta, R5Meta), axis=0)

print(midPelvis)
#print(RmidMeta)

[-0.07309568  0.0264776   0.        ]
